In [ ]:
from non_dominated_sort import fast_non_dominated_sort as fnds
from non_dominated_sort import dominance_check_between_two_points as dc
from non_dominated_sort import insert as insert
from non_dominated_sort import non_dominated_sorting as nds
from fitness_calculation import calculate_error_rate as error
from one_bit_purifying_search import one_bit_purifying_search as obps

In [ ]:
import pandas as pd
import io
from sklearn.model_selection import train_test_split
import random
from sklearn import preprocessing

import math
from operator import itemgetter

In [ ]:
pd.set_option("display.max_columns",100)

In [ ]:
df = pd.read_csv('training.csv', sep=',', encoding='utf-8', index_col=False)
df_test = pd.read_csv('test.csv', sep=',', encoding='utf-8', index_col=False)
df.head(2)

In [ ]:
df = df.drop('Unnamed: 0', axis=1)
df_test = df_test.drop('Unnamed: 0', axis=1)
df.head(2)

In [ ]:
df.shape

In [ ]:
df['target'] = df['Creditability']
df = df.drop(['Creditability'], axis=1)

In [ ]:
df_test['target'] = df_test['Creditability']
df_test = df_test.drop(['Creditability'], axis=1)

In [ ]:
print(df.shape)
print(df_test.shape)

In [ ]:
df['target'].value_counts()

In [ ]:
df_test['target'].value_counts()

In [ ]:
df.head(1)

# Creating dummy variables

In [ ]:
train_up = pd.get_dummies(df, columns = ['Account.Balance', 'Payment.Status.of.Previous.Credit', 'Purpose', 'Sex...Marital.Status', 'Guarantors', 'Most.valuable.available.asset', 'Concurrent.Credits', 'Type.of.apartment', 'Occupation', 'Foreign.Worker', 'Telephone'], drop_first= True)
test_up = pd.get_dummies(df_test, columns = ['Account.Balance', 'Payment.Status.of.Previous.Credit', 'Purpose', 'Sex...Marital.Status', 'Guarantors', 'Most.valuable.available.asset', 'Concurrent.Credits', 'Type.of.apartment', 'Occupation', 'Foreign.Worker', 'Telephone'], drop_first= True)


In [ ]:
print(train_up.shape)
print(test_up.shape)

In [ ]:
train_up_null_count = train_up.isna().sum()
train_up_null_count = train_up_null_count[train_up_null_count>0]
train_up_null_count

In [ ]:
train_up = train_up.dropna()

In [ ]:
train_up.shape

In [ ]:
train_up.head(1)

In [ ]:
test_up_null_count = test_up.isna().sum()
test_up_null_count = test_up_null_count[test_up_null_count>0]
test_up_null_count

In [ ]:
test_up = test_up.dropna()

In [ ]:
test_up.shape

In [ ]:
test_up.head(1)

# Creating training test split

In [ ]:
X_train = train_up.drop('target', axis=1)
y_train = df['target']

X_test = test_up.drop('target', axis=1)
y_test = df_test['target']

In [ ]:
X_test.columns

In [ ]:
X_train.shape

In [ ]:
X_test.shape

In [ ]:
y_train.value_counts()

In [ ]:
y_test.value_counts()

In [ ]:
def select_population_for_next_gen(population_size, P_t_plus_1_vector_fitness): 
    print("population_size" + str(population_size))
    print("len of P_t_plus_1_vector_fitness" + str(len(P_t_plus_1_vector_fitness)))
    P_t_plus_1 =  list(map(itemgetter(0), P_t_plus_1_vector_fitness))
    P_t_plus_1_fitness = list(map(itemgetter (1,2), P_t_plus_1_vector_fitness))
    P_t_plus_1_fitness = [list (item) for item in P_t_plus_1_fitness]
    
    sorted_index, sorted_values = fnds(data = P_t_plus_1_fitness)
    print("sorted_index")
    print(sorted_index)
    index_to_go_for_next_gen  = []
    
    for i in range(len(sorted_index)-1):
        if (len(sorted_index[i]) <= population_size - len(index_to_go_for_next_gen)):
            print("first if")
            print(len(sorted_index[i]))
            print(population_size - len(index_to_go_for_next_gen))
            index_to_go_for_next_gen = index_to_go_for_next_gen + sorted_index[i] 
        elif(len(index_to_go_for_next_gen) < population_size):
            places_left = population_size - len(index_to_go_for_next_gen)
            print("places_left")
            print(places_left)
            index_to_go_for_next_gen = index_to_go_for_next_gen + sorted_index[i][0:places_left] 
        else:
            print("neithr if")
    print("vedved index_to_go_for_next_gen")
    print(index_to_go_for_next_gen)
    population_for_next_gen = [P_t_plus_1_vector_fitness[i] for i in index_to_go_for_next_gen] 
    print("loop population_for_next_gen")
    print(population_for_next_gen)
    return population_for_next_gen

In [ ]:
def top_front(P_t_plus_1_vector_fitness): 
    print("P_t_plus_1_vector_fitness")
    print(P_t_plus_1_vector_fitness)
    P_t_plus_1 = list(map(itemgetter(0), P_t_plus_1_vector_fitness))
    P_t_plus_1_fitness = list(map(itemgetter(1,2), P_t_plus_1_vector_fitness))
    P_t_plus_1_fitness = [list(item) for item in P_t_plus_1_fitness]
    
    print("P_t_plus_1_fitness")
    print(P_t_plus_1_fitness)
    print("before fnds")
    sorted_index, sorted_values = fnds(data = P_t_plus_1_fitness)
    print("after fnds")
    sorted_first_front_by_error = sorted(sorted_values[0], key = lambda x: x[0], reverse=True) 
    index_of_minimum_error_element = sorted_index[0][sorted_values[0].index(sorted_first_front_by_error[0])] 
     
    top_front = [P_t_plus_1[i] for i in sorted_index[0]]
    optimal_sol =  P_t_plus_1[index_of_minimum_error_element]
       
    return top_front

In [ ]:
def optimal_sol(P_t_plus_1_vector_fitness): 
    P_t_plus_1 = list(map(itemgetter(0), P_t_plus_1_vector_fitness))
    P_t_plus_1_fitness = list(map(itemgetter(1,2), P_t_plus_1_vector_fitness))
    P_t_plus_1_fitness = [list(item) for item in P_t_plus_1_fitness]
    
    sorted_index, sorted_values = fnds(data = P_t_plus_1_fitness)
    sorted_first_front_by_error = sorted(sorted_values[0], key = lambda x: x[0], reverse=True) 
    index_of_minimum_error_element = sorted_index[0][sorted_values[0].index(sorted_first_front_by_error[0])] 
    
    top_front = [P_t_plus_1[i] for i in sorted_index[0]]
    optimal_sol =  P_t_plus_1[index_of_minimum_error_element]
    optimal_sol_fitness =  P_t_plus_1_fitness[index_of_minimum_error_element]
    
    return optimal_sol, optimal_sol_fitness

In [ ]:
def MOFS(gen_max, obps_freq, dataset_train, target_train, dataset_test, target_test, initial_population):
    P_0 = initial_population
    all_gen_population_and_fitness = []
    num_feature = 27
    
    for t in (range(gen_max)):
        print("Attention - Generation " +str(t+1) + " starts")
        P_t_plus_1 = []
        P_t_plus_1_vector_fitness = []
        P_t_plus_1_vector = []
        P_t_plus_1_fitness = []
        round_robin_flag = 0
        N = len(P_0)
        print("NNNNNNNNNNNNNNNNNNNNNN")
        print(N)
        
        for i in range(1,N+1):
            random.seed(t+30*i)
            r1, r2, r3 = random.sample(range(0,N), 3)
            print(r1, r2, r3)
            print(P_0[r1],P_0[r2],P_0[r3])
            X_r1, num_feature_r1, f1_score_r1 = error(dataset_train=X_train, target_train = y_train, dataset_test = X_test, target_test = y_test, X_rn = P_0[r1])
            X_r2, num_feature_r2, f1_score_r2 = error(dataset_train=X_train, target_train = y_train, dataset_test = X_test, target_test = y_test, X_rn = P_0[r2])
            X_r3, num_feature_r3, f1_score_r3 = error(dataset_train=X_train, target_train = y_train, dataset_test = X_test, target_test = y_test, X_rn = P_0[r3])
            
            Dict_vector_cardinality_accuracy = {(num_feature_r1, f1_score_r1):X_r1, (num_feature_r2, f1_score_r2):X_r2, (num_feature_r3, f1_score_r3):X_r3 }
            num_feature_and_accuracy_for_3_candidate_sol = [[num_feature_r1, f1_score_r1], [num_feature_r2, f1_score_r2], [num_feature_r3, f1_score_r3]]
            
            res, smallest_key = nds(num_feature_and_accuracy_for_3_candidate_sol)
            res_smallest_key = res[smallest_key]
            
            res_smallest_key = sorted(res_smallest_key, key=lambda x:x[1], reverse = False)
            round_robin_flag = round_robin_flag + 1
            
            if(res_smallest_key[0] + res_smallest_key[0] != 0):
                X_best_t = res_smallest_key[0]
            else:
                X_best_t = res_smallest_key[1]
            
            removed_ = Dict_vector_cardinality_accuracy[tuple(X_best_t)]
            
            Dict_vector_cardinality_accuracy_after_removing_best = Dict_vector_cardinality_accuracy.copy()
            Dict_vector_cardinality_accuracy_after_removing_best.pop(tuple(X_best_t))
            
            list_of_Dict_vector_cardinality_accuracy_after_removing_best = list(Dict_vector_cardinality_accuracy_after_removing_best.items())
            
            if(len(list_of_Dict_vector_cardinality_accuracy_after_removing_best) == 0):
                X_r1_new = Dict_vector_cardinality_accuracy[tuple(X_best_t)]
                X_r2_new = Dict_vector_cardinality_accuracy[tuple(X_best_t)]
            elif(len(list_of_Dict_vector_cardinality_accuracy_after_removing_best) == 1):
                X_r1_new = list_of_Dict_vector_cardinality_accuracy_after_removing_best[0][1]
                X_r2_new = list_of_Dict_vector_cardinality_accuracy_after_removing_best[0][1]
            else:
                X_r1_new = list_of_Dict_vector_cardinality_accuracy_after_removing_best[0][1]
                X_r2_new = list_of_Dict_vector_cardinality_accuracy_after_removing_best[1][1]
                
            base_vector = Dict_vector_cardinality_accuracy[tuple(X_best_t)]
            X_i = P_0[i-1]
            
            X_ri, num_feature_ri, f1_score_ri = error(dataset_train=X_train, target_train = y_train, dataset_test = X_test, target_test = y_test, X_rn = X_i)
            X_rbase, num_feature_rbase, f1_score_rbase = error(dataset_train=X_train, target_train = y_train, dataset_test = X_test, target_test = y_test, X_rn = base_vector)
            
            xi_dominates_base, base_dominates_xi, base_and_xi_are_nondominating = dc(point1=[num_feature_rbase, f1_score_rbase], point2 =[num_feature_ri, f1_score_ri])
            
            X_r1_for_xor = X_r1_new.copy()
            X_r2_for_xor = X_r2_new.copy()
            
            sigma = 0.01
            c_i = [0]*num_feature
            v_i = [0]*num_feature
            F=0.1
            random.seed(t+30*i)
            random_val = random.sample(range(0,100), 1)
            random_value = random_val[0]/100
            
            if(base_dominates_xi == True):
                c_i = [sigma]*num_feature
            else:
                for i in range(num_feature):
                    c_i[i] = min(1, (F*int((bool(X_r1_for_xor[i]) ^ bool(X_r2_for_xor[i])))+sigma))
            mu = math.exp(-math.sqrt(t)) #t is generation count
            
            #for i in range(num_feature):
            #    v_i[i] = mu * (math.floor((X_r1[i] + X_r2[i] + X_r3[i])/1.5)) + (1-mu) *((base_vector[i] * int(bool(X_r1_new[i]) ^ bool(X_r2_new[i]))) + (int(not(bool(base_vector[i]))) * int(bool(X_r1_new[i]) & bool(X_r2_new[i]))))
            #    if(v_i[i] < 0.5):
            #        v_i[i] = 0
            #    else:
            #        v_i[i] = 1
                    
            for i in range(num_feature):
                if c_i[i] < random_value:
                    v_i[i] = base_vector[i]
                else:
                    v_i[i] = 1-base_vector[i]
                
            if sum(v_i)>0:
                pass
            else:
                v_i = base_vector
                
            random.seed(t+30*i)

            u_0_1 = random.sample(range(0,100), 1)[0]
            u_0_1 = u_0_1/100

            random.seed(t+30*i)
            h=random.sample(range(0,N),1)[0]
            cr=0.5
            u_i = [0]*num_feature
                
            for i in range(num_feature):
                if(u_0_1 < cr or i==h):
                    u_i[i] = v_i[i]
                else:
                    u_i[i] = X_i[i]
                        
            X_ui, num_feature_ui, f1_score_ui = error(dataset_train=X_train, target_train = y_train, dataset_test = X_test, target_test = y_test, X_rn = u_i)
            xi_dominates_ui, ui_dominates_xi, ui_and_xi_are_nondominating = dc(point1=[num_feature_ui, f1_score_ui], point2 =[num_feature_ri, f1_score_ri])

            if(xi_dominates_ui == 1):
                print("xi_dominates_ui")
                P_t_plus_1_vector_fitness.append([X_ri, num_feature_ri, f1_score_ri])
            elif(ui_dominates_xi == 1):
                print("ui_dominates_xi")
                P_t_plus_1_vector_fitness.append([X_ui, num_feature_ui, f1_score_ui])
            else:
                print("xi ui nondominating")
                P_t_plus_1_vector_fitness.append([X_ri, num_feature_ri, f1_score_ri])
                P_t_plus_1_vector_fitness.append([X_ui, num_feature_ui, f1_score_ui])
        
        print("P_t_plus_1_vector_fitnessP_t_plus_1_vector_fitnessP_t_plus_1_vector_fitnessP_t_plus_1_vector_fitness")
        print(P_t_plus_1_vector_fitness)
        
        #Remove duplicates
        P_t_plus_1_vector_fitness_unique=[]
        for i in P_t_plus_1_vector_fitness:
            if i not in P_t_plus_1_vector_fitness_unique:
                P_t_plus_1_vector_fitness_unique.append(i)
                
        dup_count = len(P_t_plus_1_vector_fitness) - len(P_t_plus_1_vector_fitness_unique)
        #print(dup_count)
        
        print("P_t_plus_1_vector_fitness")
        print(P_t_plus_1_vector_fitness)
        print("P_t_plus_1_vector_fitness_unique")
        print(P_t_plus_1_vector_fitness_unique)
        
        P_t_plus_1_vector_fitness = P_t_plus_1_vector_fitness_unique.copy()
        
        if(gen_max/(obps_freq+1) == math.ceil(gen_max/(obps_freq+1))):
            top_front_ = top_front(P_t_plus_1_vector_fitness=P_t_plus_1_vector_fitness)
            print("top_front_")
            print(top_front_)
            optimal_sol_, optimal_sol_fitness_ = optimal_sol(P_t_plus_1_vector_fitness=P_t_plus_1_vector_fitness)
            obps_result = obps(X_train, X_test, y_train, y_test, S=top_front_, optimal_sol = optimal_sol_)
                        
            to_be_removed_sol = optimal_sol_.copy()
            to_be_removed_fit = optimal_sol_fitness_.copy()
                       
            to_be_removed_sol = [to_be_removed_sol, to_be_removed_fit[0], to_be_removed_fit[1]]
            
            P_t_plus_1_vector_fitness.remove(to_be_removed_sol)
            P_t_plus_1_vector_fitness.append(obps_result[0])
        print("vedP_t_plus_1_vector_fitness")
        print(P_t_plus_1_vector_fitness)
        print("len of P_t_plus_1_vector_fitness")
        print(len(P_t_plus_1_vector_fitness))
        print(N)
        if len(P_t_plus_1_vector_fitness) > N:
            population_for_next_gen_ = select_population_for_next_gen(population_size=N, P_t_plus_1_vector_fitness=P_t_plus_1_vector_fitness)
        else:
            population_for_next_gen_ = P_t_plus_1_vector_fitness
        
        print("population_for_next_gen_population_for_next_gen_")
        print(population_for_next_gen_)
        all_gen_population_and_fitness.append(population_for_next_gen_)
        
        P_0 = list(map(itemgetter(0), population_for_next_gen_))
        
    return P_t_plus_1_vector_fitness, all_gen_population_and_fitness

In [ ]:
import numpy as np
import random
Feature_population = []
rng = np.random.default_rng(27) #number of features
for i in range(100): #number of candidate solution in initial population
    array_of_0_1 = rng.random(27) #number of features
    array_of_0_1[array_of_0_1>=0.5] = 1
    array_of_0_1[array_of_0_1<0.5] = 0
    array_of_0_1=array_of_0_1.astype(int)
    list_of_0_1 = list(array_of_0_1)
    
    Feature_population.append(list_of_0_1)

In [ ]:
len(Feature_population)

In [ ]:
print(Feature_population)

In [ ]:
last_gen_vector_fitness, all_generation_population_and_fitness = MOFS(gen_max=200, obps_freq=1, dataset_train=X_train, target_train = y_train, dataset_test = X_test, target_test = y_test, initial_population = Feature_population)

In [ ]:
print(all_generation_population_and_fitness)